In [ ]:
import pandas as pd
import re
from Bio.SeqIO.FastaIO import SimpleFastaParser
import gzip

# Function to return the count number  in the fasta in a dictionary
def lncRNA_db(file_path, separator, RNA_index):
    lncRNA_dictionary = {}
    with gzip.open(file_path, 'r') as fasta_file:
        fasta_contents = fasta_file.readlines()
        for line in fasta_contents:
            if line.startswith(">"):
                name = line.split(separator)[RNA_index]
                if name not in lncRNA_dictionary:
                    lncRNA_dictionary[name] = 1
                else:
                    lncRNA_dictionary[name] += 1
    return lncRNA_dictionary

# Function to read fasta and return a DataFrame
def read_fasta(file_path, columns):
    file_path = "C:/Users/Swadha/Desktop/INTERNSHIP 2024/data collection/lncRNA_LncBookv2.0.fa.gz"
   
    bd_func = lncRNA_db(file_path, '|', 1)  # Execute function
    
    with gzip.open(file_path, 'rt') as fasta_file:
        records = []  # Create empty list
        for title, sequence in SimpleFastaParser(fasta_file):
            record = []
            title_splits = re.findall(r"[\w']+", title)  # Data cleaning
            
            record.append(title_splits[0])  # First values are ID
            record.append(len(sequence))  # Second values are sequence lengths
            sequence = " ".join(sequence)  # It converts into one line
            record.append(sequence)  # Third values are sequences

            
            records.append(record)  # Add the record to the list

    return pd.DataFrame(records, columns=columns)  # Return DataFrame

# Example usage
file_path = "lncRNA_LncBookv2.0.fa.gz"
columns = ["id", "sequence_length", "sequence"]
data = read_fasta(file_path, columns)

print(data)
